In [1]:
import torch
from torch.autograd import Variable as var
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = x.view(-1, self.num_features(x.size()[1:]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x
    
    def num_features(self, sz):
        n_features = 1
        for s in sz:
            n_features *= s
        return n_features
    
        

In [3]:
conv_net = Net()
print(conv_net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### Learnable parameters

In [4]:
params = list(conv_net.parameters())

In [5]:
print(len(params))

10


In [6]:
for i in range(10):
    print(params[i].size())

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


#### Assuming 32x32 input size

In [7]:
net_input = var(torch.randn(1, 1, 32, 32), requires_grad=True)

In [8]:
out = conv_net(net_input)

In [9]:
print(out)

Variable containing:
1.00000e-02 *
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  5.6174  5.0602  2.0968  0.0000
[torch.FloatTensor of size 1x10]



In [10]:
conv_net.zero_grad()
out.backward(torch.rand(1,10))

In [11]:
output = conv_net(net_input)
target = var(torch.arange(1, 11))  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.3033
[torch.FloatTensor of size 1]



In [12]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0])

In [13]:
conv_net.zero_grad()
print(conv_net.conv1.bias.grad)

loss.backward()

print(conv_net.conv1.bias.grad)

Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

Variable containing:
1.00000e-02 *
 -1.9503
  3.3265
 -5.3848
 -1.2597
 -0.1662
  4.3948
[torch.FloatTensor of size 6]



In [14]:
import torch.optim as optim

In [15]:
optimizer = optim.SGD(conv_net.parameters(), lr=0.01)

In [16]:
optimizer.zero_grad()

In [17]:
output = conv_net(net_input)

In [18]:
loss = criterion(output, target)

In [19]:
loss.backward()

In [20]:
optimizer.step()